In [38]:
import pandas as pd
import numpy as np
from glob import glob
import re

In [39]:
df = pd.read_json('data/ucsb_speeches_2016.json')

In [40]:
df.head()

,link,title,date,person,transcript
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,\nHello Grand Valley! Thank you! Thank you.\nI...
3,/documents/remarks-js-dorton-arena-raleigh-nor...,"Remarks at J.S Dorton Arena in Raleigh, North ...",2016-11-07 00:00:00+00:00,Donald J. Trump,\nThis is amazing. What a great place. Thank y...
4,/documents/remarks-north-carolina-state-univer...,Remarks at North Carolina State University in ...,2016-11-07 00:00:00+00:00,Hillary Clinton,\nThank you all! Whoa! Thank you! Thank you. W...
5,/documents/remarks-the-university-pittsburgh,Remarks at the University of Pittsburgh,2016-11-07 00:00:00+00:00,Hillary Clinton,"\nHello, Pittsburgh! Woah! Hello back there!\n..."
6,/documents/remarks-eastern-market-detroit-mich...,"Remarks at Eastern Market in Detroit, Michigan",2016-11-04 00:00:00+00:00,Hillary Clinton,"\nHello Detroit! Thank you! I'll tell you, I a..."


In [41]:
df['transcript'].str.split("[.!?]").loc[2]

['\nHello Grand Valley',
 ' Thank you',
 ' Thank you',
 '\nIt is great to be back in western Michigan',
 ' Thank you',
 ' Are there any Lakers here',
 '\nOh, my gosh',
 ' I am so thrilled to be here',
 ' I want to thank the president of this great university, Tom Haas, the faculty, the staff, the students',
 ' I want to thank Brandon Dillon, the Chair of the Michigan Democratic Party',
 ' And I especially really want to thank you for sending Debbie Stabenow to the United States Senate',
 "\nI have also the great personal experience and privilege of having worked with Debbie, gotten to know her, become a friend of hers, and I am so excited at the possibility that I'll be able to work with her again",
 " But I want you to know this - and I really, really hope you'll tell folks, because there is no more dedicated, effective member of the United States Senate than Debbie Stabenow",
 '\nShe fights for all of you every day, but she does it with that smile that never leaves her face',
 ' And 

In [42]:
#df = df.reset_index().set_index(['link','title','date','person'])
df['transcript'] = df['transcript'].str.split("[.!?]")
df['transcript'] = df['transcript'].apply(lambda x: [(i,v) for (i,v) in enumerate(x)])

In [43]:
df = df.explode('transcript')
df.head()

,link,title,date,person,transcript
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,"(0, \nHello Grand Valley)"
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,"(1, Thank you)"
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,"(2, Thank you)"
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,"(3, \nIt is great to be back in western Michigan)"
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,"(4, Thank you)"


In [47]:
df['sent'] = df['transcript'].apply(lambda x: x[0])
df['str'] = df['transcript'].apply(lambda x: x[1])

In [55]:
# remove leading/trailing whitespaces
df['str'] = df['str'].str.strip()
# drop 0-length strings 
df = df[df['str'].str.len() > 0]
# drop the intermediate column
df = df.drop(columns='transcript')

In [65]:
df.head()

,link,title,date,person,sent,str
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,0,Hello Grand Valley
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,1,Thank you
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,2,Thank you
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,3,It is great to be back in western Michigan
2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton,4,Thank you


In [70]:
library = df.reset_index()[['index','link','title','date','person']].drop_duplicates()
library = library.rename({'index':'speech_id'},axis=1)

In [78]:
library.head()

,speech_id,link,title,date,person
0,2,/documents/remarks-grand-valley-state-universi...,Remarks at Grand Valley State University in Gr...,2016-11-07 00:00:00+00:00,Hillary Clinton
252,3,/documents/remarks-js-dorton-arena-raleigh-nor...,"Remarks at J.S Dorton Arena in Raleigh, North ...",2016-11-07 00:00:00+00:00,Donald J. Trump
1049,4,/documents/remarks-north-carolina-state-univer...,Remarks at North Carolina State University in ...,2016-11-07 00:00:00+00:00,Hillary Clinton
1160,5,/documents/remarks-the-university-pittsburgh,Remarks at the University of Pittsburgh,2016-11-07 00:00:00+00:00,Hillary Clinton
1323,6,/documents/remarks-eastern-market-detroit-mich...,"Remarks at Eastern Market in Detroit, Michigan",2016-11-04 00:00:00+00:00,Hillary Clinton


In [88]:
df = df.reset_index()[['index','sent','str']].rename({'index':'speech_id'},axis=1)
df.head()

,speech_id,sent,str
0,2,0,Hello Grand Valley
1,2,1,Thank you
2,2,2,Thank you
3,2,3,It is great to be back in western Michigan
4,2,4,Thank you


In [80]:
# Hmm, do the transcripts indicate paragraphs in some way? Like a pause, or something? I think these ones might, but the youtube ones might not.

In [81]:
import nltk

In [94]:
token = df.set_index(['speech_id','sent'])['str'].apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
    .stack().to_frame()\
    .rename(columns={0:'pos_tuple'})
token['pos'] = token.pos_tuple.apply(lambda x: x[1])
token['token_str'] = token.pos_tuple.apply(lambda x: x[0])
token = token.drop('pos_tuple', 1)

In [95]:
token.index.names = ['speech_id','sent_id','token_id']

In [96]:
token.head()

pos token_str
speech_id sent_id token_id               
2         0       0         NNP     Hello
                  1         NNP     Grand
                  2         NNP    Valley
          1       0          NN     Thank
                  1         PRP       you

In [97]:
# Lowercase, remove non-word characters
token['term_str'] = token['token_str'].str.lower().str.replace('[\W_]', '')

In [99]:
# Checkpoint
token.head(20)

pos token_str  term_str
speech_id sent_id token_id                         
2         0       0         NNP     Hello     hello
                  1         NNP     Grand     grand
                  2         NNP    Valley    valley
          1       0          NN     Thank     thank
                  1         PRP       you       you
          2       0          NN     Thank     thank
                  1         PRP       you       you
          3       0         PRP        It        it
                  1         VBZ        is        is
                  2          JJ     great     great
                  3          TO        to        to
                  4          VB        be        be
                  5          RB      back      back
                  6          IN        in        in
                  7          JJ   western   western
                  8         NNP  Michigan  michigan
          4       0          NN     Thank     thank
                  1         PRP       you       you
          5       0         NNP       Are       are
                  1          EX     there     there